# Try using CoAtNet w/ Breast Histopathology Images

* The original dataset consisted of 162 whole mount slide images of Breast Cancer (BCa) specimens scanned at 40x. From that, 277,524 patches of size 50 x 50 were extracted (198,738 IDC negative and 78,786 IDC positive). Each patch’s file name is of the format: uxXyYclassC.png — > example 10253idx5x1351y1101class0.png . Where u is the patient ID (10253idx5), X is the x-coordinate of where this patch was cropped from, Y is the y-coordinate of where this patch was cropped from, and C indicates the class where 0 is non-IDC and 1 is IDC.

## 1) Get Image Data & Data Preprocessing

In [28]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm

In [112]:
path = "/home/bis/211105_CSH_MPL/kaggleBreast/"

ids = os.listdir(path)
paths_1 = []
paths_0 = []
for id in tqdm(ids):
    try:
        files1 = os.listdir(path + id + '/1/')
        files0 = os.listdir(path + id + '/0/')
        for x in files1:
            paths_1.append(path + id + '/1/' + x)
        for x in files0:
            paths_0.append(path + id + '/0/' + x)
    except:
        FileNotFoundError
print(len(paths_1))
print(len(paths_0))
print(len(paths_0+paths_1))

  0%|          | 0/280 [00:00<?, ?it/s]

78786
198738
277524


In [136]:
#train test split
#NPHD - positive 5000, negative 5000

import random

random.shuffle(paths_1)
random.shuffle(paths_0)
train_paths = paths_0[:200] + paths_1[:200]
valid_paths = paths_0[200:250]+paths_1[200:250]
test_paths = paths_0[250:500]+paths_1[250:500]
print(len(train_paths))
print(len(valid_paths))
print(len(test_paths))

400
100
500


In [137]:
#image to array
#processing 277524 imgs take about 10 mins
#processing 10000 imgs take about 1 mins
import keras_preprocessing.image as IMAGE
import PIL
from PIL import Image
import torch
from torch.utils.data import  TensorDataset, DataLoader
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize([64, 64]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

train_images=[]
for i in tqdm(train_paths):
    label = int(i[-5]) # class1.png --> 1
    img = transform(IMAGE.load_img(i, target_size=(64,64)))
    train_images.append((img,label))

valid_images=[]
for i in tqdm(valid_paths):
    label = int(i[-5]) # class1.png --> 1
    img = transform(IMAGE.load_img(i, target_size=(64,64)))
    valid_images.append((img,label))    

test_images=[]
for i in tqdm(test_paths):
    label = int(i[-5]) # class1.png --> 1
    img = transform(IMAGE.load_img(i, target_size=(64,64)))
    test_images.append((img,label))
    

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [138]:
print("train img #",len(train_images))
print("test img #",len(test_images))

train img # 400
test img # 500


In [139]:
from torch.utils.data import  TensorDataset, DataLoader

dataloaders = {}
dataloaders['train'] = DataLoader(train_images, batch_size=16, shuffle=True)
dataloaders['valid'] = DataLoader(valid_images, batch_size=len(valid_images), shuffle=True)
dataloaders['test'] = DataLoader(test_images, batch_size=len(test_images), shuffle=True)

## 2) Try CoAtNet

def coatnet_0():<br/>
    num_blocks = [2, 2, 3, 5, 2]            # L<br/>
    channels = [64, 96, 192, 384, 768]      # D<br/>
    return CoAtNet((224, 224), 3, num_blocks, channels, num_classes=1000)<br/>

<br/>
def coatnet_1():<br/>
    num_blocks = [2, 2, 6, 14, 2]           # L<br/>
    channels = [64, 96, 192, 384, 768]      # D<br/>
    return CoAtNet((224, 224), 3, num_blocks, channels, num_classes=1000)<br/>

<br/>
def coatnet_2():<br/>
    num_blocks = [2, 2, 6, 14, 2]           # L<br/>
    channels = [128, 128, 256, 512, 1026]   # D<br/>
    return CoAtNet((224, 224), 3, num_blocks, channels, num_classes=1000)<br/>

<br/>
def coatnet_3():<br/>
    num_blocks = [2, 2, 6, 14, 2]           # L<br/>
    channels = [192, 192, 384, 768, 1536]   # D<br/>
    return CoAtNet((224, 224), 3, num_blocks, channels, num_classes=1000)<br/>


def coatnet_4():<br/>
    num_blocks = [2, 2, 12, 28, 2]          # L<br/>
    channels = [192, 192, 384, 768, 1536]   # D<br/>
    return CoAtNet((224, 224), 3, num_blocks, channels, num_classes=1000)<br/>

In [140]:
# Print total batches
total_batches = len(dataloaders["train"])
print(f"Total batches: {total_batches}")

Total batches: 25


In [144]:
# Init CoAtNet model
import torch.nn as nn
from coatnet import coatnet_0,coatnet_1,coatnet_2,coatnet_3,coatnet_4
from coatnet import CoAtNet


#coatnet_4
num_blocks = [2, 2, 3, 5, 2]            # L
channels = [64, 96, 192, 384, 768]      # D
block_types=['C', 'C', 'T', 'T']        # 'C' for MBConv, 'T' for Transformer
model = CoAtNet((64,64), 3, num_blocks, channels, num_classes = 2) 

#loss - cross entropy
#optimizer - adam
# loss_fn = nn.MSELoss()
loss_fn = nn.CrossEntropyLoss()

# Define Optimizer - 이해 필요
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=0.0001,
    betas=(0.9, 0.999),
    eps=10e-8,
    amsgrad=False,
)

# Print model info
print(model)

CoAtNet(
  (s0): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU()
    )
    (1): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU()
    )
  )
  (s1): Sequential(
    (0): MBConv(
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (proj): Conv2d(64, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (conv): PreNorm(
        (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (fn): Sequential(
          (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [145]:
def div(i1, i2):
    return i1/i2 if i2 else 0

In [146]:
# Train CoAtNet
train = {'loss':[]}
valid = {'loss':[],'accuracy':[],'specificity':[],'recall':[],'precision':[],'negpred':[],'f1':[]}

epochs = 20 
for eid in range(epochs):
    # Logging
    print("Epoch {}".format(eid))
    loss_avg = 0
    for i, (inputs, targets) in enumerate(dataloaders["train"]):
        # Train model
        model.train()
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss_avg += loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()   
        # Logging
    train['loss'].append(loss_avg/total_batches)
    print("Training loss: {}".format(loss_avg/total_batches))
    # Validate model
    model.eval()
    with torch.no_grad():
        TP = 0
        FP = 0
        TN = 0
        FN = 0
        total = 0
        loss_avg = 0
        for i, (inputs, targets) in enumerate(dataloaders["valid"]):
            outputs = model(inputs)
            loss_avg += loss_fn(outputs, targets)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            TP += ((predicted == targets)&(predicted == 1)).sum().item()
            FP += ((predicted != targets)&(targets == 0)).sum().item()
            TN += ((predicted == targets)&(predicted == 0)).sum().item()
            FN += ((predicted != targets)&(targets == 1)).sum().item()
        acc = div((TP+TN),total)
        spec = div(TN,(TN+FP))
        negpred = div(TN,(TN+FN))
        rec = div(TP,(TP+FN))
        prec = div(TP,(TP+FP))
        f1 = div(2*(prec*rec),(prec+rec))
        valid['accuracy'].append(acc) 
        valid['specificity'].append(spec) 
        valid['recall'].append(rec) 
        valid['precision'].append(prec) 
        valid['negpred'].append(negpred) 
        valid['f1'].append(f1) 
        valid['loss'].append(loss_avg/total_batches) 
        print("Validation loss: {}".format(loss_avg/total_batches))
        print("Validation accuracy: {}".format(acc))
        

Epoch 0
Training loss: 1.0034186840057373
Validation loss: 0.03731130436062813
Validation accuracy: 0.5
Epoch 1
Training loss: 0.41606879234313965
Validation loss: 0.01618383266031742
Validation accuracy: 0.86
Epoch 2
Training loss: 0.36123818159103394
Validation loss: 0.014746812172234058
Validation accuracy: 0.86
Epoch 3
Training loss: 0.26743653416633606
Validation loss: 0.020006917417049408
Validation accuracy: 0.83
Epoch 4
Training loss: 0.22217343747615814
Validation loss: 0.032366249710321426
Validation accuracy: 0.79
Epoch 5
Training loss: 0.1629261076450348
Validation loss: 0.023033270612359047
Validation accuracy: 0.87
Epoch 6
Training loss: 0.07543262094259262
Validation loss: 0.020127832889556885
Validation accuracy: 0.85
Epoch 7
Training loss: 0.032308418303728104
Validation loss: 0.03599844127893448
Validation accuracy: 0.79
Epoch 8
Training loss: 0.02464905008673668
Validation loss: 0.03917780518531799
Validation accuracy: 0.82
Epoch 9
Training loss: 0.11016647517681122


In [147]:
TP = 0
FP = 0
TN = 0
FN = 0
total = 0
for i, (inputs, targets) in enumerate(dataloaders["test"]):
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += targets.size(0)
    correct += (predicted == targets).sum().item()
    TP += ((predicted == targets)&(predicted == 1)).sum().item()
    FP += ((predicted != targets)&(targets == 0)).sum().item()
    TN += ((predicted == targets)&(predicted == 0)).sum().item()
    FN += ((predicted != targets)&(targets == 1)).sum().item()
acc = div((TP+TN),total)
spec = div(TN,(TN+FP))
negpred = div(TN,(TN+FN))
rec = div(TP,(TP+FN))
prec = div(TP,(TP+FP))
f1 = div(2*(prec*rec),(prec+rec))

In [148]:
#CE, lr 0.00001, 
print("Test accuracy: {}".format(round(acc,10)))
print("Test specifity: {}".format(round(spec,10)))
print("Test negative predicable: {}".format(round(negpred,10)))
print("Test recall: {}".format(round(rec,10)))
print("Test precision: {}".format(round(prec,10)))
print("Test f1: {}".format(round(f1,10)))
print("TP:",TP)
print("FP:",FP)
print("TN:",TN)
print("FN:",FN)

Test accuracy: 0.806
Test specifity: 0.84
Test negative predicable: 0.7865168539
Test recall: 0.772
Test precision: 0.8283261803
Test f1: 0.7991718427
TP: 193
FP: 40
TN: 210
FN: 57


In [132]:
#CE, lr 0.0003, 
print("Test accuracy: {}".format(round(acc,10)))
print("Test specifity: {}".format(round(spec,10)))
print("Test negative predicable: {}".format(round(negpred,10)))
print("Test recall: {}".format(round(rec,10)))
print("Test precision: {}".format(round(prec,10)))
print("Test f1: {}".format(round(f1,10)))

Test accuracy: 0.792
Test specifity: 0.676
Test negative predicable: 0.8802083333
Test recall: 0.908
Test precision: 0.737012987
Test f1: 0.8136200717


In [128]:
print("TP:",TP)
print("FP:",FP)
print("TN:",TN)
print("FN:",FN)

TP: 227
FP: 81
TN: 169
FN: 23
